In [46]:
import httpx, re
import pandas as pd
from bs4 import BeautifulSoup
from itertools import product


# 关键词列表
keywords_list = [
    "营销策划", "异业联盟", "薪酬自己定", "新媒体", "项目互换", "文案", "推广", "头条",
    "私人定制", "社交", "商务合作", "沙龙", "软件推广", "人脉", "圈子", "对接人",
    "求职", "竞价", "sem", "SEM", "排名", "媒体合作", "裂变", "客户管理", "客户成功",
    "绝活", "俱乐部", "竞价数据分析", "竞价", "精准引流", "精准营销", "今日头条", "兼职讲师",
    "兼职工作", "激活休眠", "获客", "伙伴", "互推", "互联网渠道交流", "公众号", "公司初创",
    "干货", "对接", "独特", "抖音", "董事长", "电销", "电商", "电话回访", "弹性工作",
    "大数据", "大神", "大客户ka", "大客户", "大会", "创业中", "创新类产品", "传媒公司",
    "传播公司", "承接销售", "承接电话销售", "销售总监", "独家资源", "一手资源", "成功案例",
    "产品置换", "策划", "找工作"
]

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',
    'cookie': 'ZFY=97quXcD50442KxUKCT:BP2Y5a01LDSQaqIdy1BfWPw:BM:C; BAIDUID=E6611FA073EE10B08D094FC89F3D85C5:FG=1; H_WISE_SIDS=110085_299594_298696_301025_607111_607770_307086_277936_611621_612162_613051_613583_610631_614666_614852_614884_615078_613117_609499_615097_615211_614025_615383_615414_614430_615527_615575_612711_614509_615749_615769_615726_612886_615840_615676_614596_615705_615978_615159_614396_616120_614436_616196_616210_616158_616313_616315_616332_616376_615838_614438_616512_616523_616576_616611_616624_614837_615165_616800_612952_616900_616952_616483_616566_617017; rsv_i=a6d0RCU44Zz9lJjVoIi6T+9OLYonW/iNG1Ypi/DMScQzdBGVnDvTdspcKGz9POoIVCBwfZC26gfwIrqca7qPmsqiyvrX8+c; BA_HECTOR=2424a1a0a08l2l0g240g84ag9a4plj1jbueqo1u; plus_lsv=397074fef83b0eda; BDORZ=AE84CDB3A529C0F8A2B9DCDD1D18B695; HMACCOUNT=CEF825C65830DFC2; plus_cv=1::m:0ae33561; bd_af=1; delPer=0; POLYFILL=0; MSA_PBT=148; MSA_ZOOM=1000; ab_sr=1.0.1_YmEwMzY0YTI4MmE1MjBmZGQyMmE1YjljNjdmNzMxYjc0NDhlZDk1MzNjMjAwNzMxZTdlZjQ1MDdmYTMyMjMwMmE0OWIwZmJiMDZhYWVkNGNhYmIyMDYyODY3YzIxZTU1YmZlMGY3Y2RhMzk5YWQ3MThkMzMyYjY1ZjExMWFiZGNmZThjMmJlODk5YzU3MjEwOTE2YjM5MjRkZDRmZWU1Yg==; PSCBD=31%3A1_25%3A1; SE_LAUNCH=5%3A28730143_16%3A28730143_31%3A28730143_25%3A28730143; H_WISE_SIDS_BFESS=110085_299594_298696_301025_607111_607770_307086_277936_611621_612162_613051_613583_610631_614666_614852_614884_615078_613117_609499_615097_615211_614025_615383_615414_614430_615527_615575_612711_614509_615749_615769_615726_612886_615840_615676_614596_615705_615978_615159_614396_616120_614436_616196_616210_616158_616313_616315_616332_616376_615838_614438_616512_616523_616576_616611_616624_614837_615165_616800_612952_616900_616952_616483_616566_617017; BAIDUID_BFESS=E6611FA073EE10B08D094FC89F3D85C5:FG=1; MSA_WH=2490_1350; MSA_PHY_WH=1440_2560; BDSVRTM=3; __bsi=11493240161070922402_00_9_R_R_7_0303_c02f_Y; COOKIE_SESSION=33_4_2_2_2_w1_0_2_1_0_0_2_4_1723813213%7C2%230_0_0_0_0_0_0_0_1723813180%7C1; wpr=0; FC_MODEL=0_0_0_0_13.57_0_0_0_0_0_24.35_0_2_81_1_74_0_0_1723813132%7C2%2313.57_0_0_2_1_0_1723813132%7C2%230_ax_1_0_0_0_0_1723813132'
}



def query_baidu(keyword, page):
    # 百度查询链接
    url = f"http://m.baidu.com/s?word={keyword}&pn={page*10}" 
    response = httpx.get(url, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')

    # 用于存储结果
    results = []

    # 记录每页的每个结果
    for idx, item in enumerate(soup.select('.result')):
        result_content = item.select_one('.c-result-content')
        if '搜索结果' in result_content.text:
            continue

        title_item = result_content.select_one('.tts-b-hl, .c-title-text, ._paragraph_1bhnz_2, .cos-text-subtitle')
        title = title_item.text if title_item else '无标题'

        link_item = item.get('data-log', None)
        link = eval(link_item)['mu'] if link_item else '无链接'

        summary = result_content.text[len(title):].strip() if title_item else '无简介'
        summary = re.sub(r'\s+', ' ', summary)

        results.append({
            '关键词': keyword,
            '百度手机页码': page + 1,
            '百度手机排名': idx + 1,
            '百度手机标题': title,
            '百度手机网址': link,
            '百度手机简介': summary
        })

    return results


In [47]:
result = query_baidu("汽车", 2)

df = pd.DataFrame(result)
df

""


In [40]:
df.head()

""


In [9]:
from my_thread import ExampleThreadManager

In [34]:
class FetchThread(ExampleThreadManager):
    def get_args(self, obj: object):
        return (obj[0], obj[1], )
    
    def process_result(self):
        result_dict = self.get_result_dict()
        result = []
        for d in result_dict.values():
            result += d
        return result
    
fetcher = FetchThread(
    query_baidu, 
    thread_num=60,
    tqdm_desc='baiduFetchProcess', 
    show_progress=True
    )

In [35]:
args_list = []

for i in product(keywords_list, range(10)):
    args_list.append(i)

args_list

[('营销策划', 0),
 ('营销策划', 1),
 ('营销策划', 2),
 ('营销策划', 3),
 ('营销策划', 4),
 ('营销策划', 5),
 ('营销策划', 6),
 ('营销策划', 7),
 ('营销策划', 8),
 ('营销策划', 9),
 ('异业联盟', 0),
 ('异业联盟', 1),
 ('异业联盟', 2),
 ('异业联盟', 3),
 ('异业联盟', 4),
 ('异业联盟', 5),
 ('异业联盟', 6),
 ('异业联盟', 7),
 ('异业联盟', 8),
 ('异业联盟', 9),
 ('薪酬自己定', 0),
 ('薪酬自己定', 1),
 ('薪酬自己定', 2),
 ('薪酬自己定', 3),
 ('薪酬自己定', 4),
 ('薪酬自己定', 5),
 ('薪酬自己定', 6),
 ('薪酬自己定', 7),
 ('薪酬自己定', 8),
 ('薪酬自己定', 9),
 ('新媒体', 0),
 ('新媒体', 1),
 ('新媒体', 2),
 ('新媒体', 3),
 ('新媒体', 4),
 ('新媒体', 5),
 ('新媒体', 6),
 ('新媒体', 7),
 ('新媒体', 8),
 ('新媒体', 9),
 ('项目互换', 0),
 ('项目互换', 1),
 ('项目互换', 2),
 ('项目互换', 3),
 ('项目互换', 4),
 ('项目互换', 5),
 ('项目互换', 6),
 ('项目互换', 7),
 ('项目互换', 8),
 ('项目互换', 9),
 ('文案', 0),
 ('文案', 1),
 ('文案', 2),
 ('文案', 3),
 ('文案', 4),
 ('文案', 5),
 ('文案', 6),
 ('文案', 7),
 ('文案', 8),
 ('文案', 9),
 ('推广', 0),
 ('推广', 1),
 ('推广', 2),
 ('推广', 3),
 ('推广', 4),
 ('推广', 5),
 ('推广', 6),
 ('推广', 7),
 ('推广', 8),
 ('推广', 9),
 ('头条', 0),
 ('头条', 1),
 ('头条', 2),
 ('头条', 3),
 ('头条', 4),


In [36]:
fetcher.start(args_list, 'serial')

baiduFetchProcess:  20%|██        | 12/60 [00:09<00:37,  1.28it/s]

KeyboardInterrupt: 

In [32]:
fetcher.handle_error()
content_list = fetcher.process_result()

df = pd.DataFrame(content_list)
df.head(5)

,关键词,百度手机页码,百度手机排名,百度手机标题,百度手机网址,百度手机简介
0,营销策划,3,1,营销策划方案(精选14篇),https://www.yjbys.com/cehuashu/2973673.html,"2023年6月16日为了确保工作或事情有序地进行,就不得不需要事先制定方案,方案是有很强可操..."
1,营销策划,3,2,营销方案策划书(精选20篇),https://www.yjbys.com/cehuashu/3624583.html,2023年8月11日以下就是我们团队准备的营销策划方案。 二、市场环境 (一)宏观环境分析 ...
2,营销策划,3,3,营销策划方案(通用21篇),https://www.yjbys.com/cehuashu/2981420.html,"2023年5月12日营销策划方案(通用21篇) 为了确保事情或工作有效开展,往往需要预先进行..."
3,营销策划,3,4,【热门】市场营销活动策划20篇,http://www.unjs.com/fanwenku/450137.html,"1月6日相信许多人会觉得策划书很难写吧,下面是小编帮大家整理的市场营销活动策划,仅供参考,欢..."
4,营销策划,3,5,营销策划方案怎么写(例文10篇),https://m.51test.net/show/11004675.html,2023年12月26日营销策划方案怎么写(例文10篇) 【#策划# #营销策划方案怎么写(例...
...,...,...,...,...,...,...
64,大会,2,6,全国苏维埃区域代表大会的历史细节,https://baijiahao.baidu.com/s?id=1792735620630...,参加全国苏维埃区域代表大会的正式代表有48人，他们来自全国各个红色根据地和游击区，作家柔石、...
65,大会,2,7,全国科技大会、国家科学技术奖励大会、两院院士大会 - 中华人民共和...,https://www.most.gov.cn/ztzl/qgkjdh2023jl/,7月21日 新华社北京6月24日电 全国科技大会、国家科学技术奖励大会和中国科学院第二十一次...
66,大会,2,8,2024世界人工智能大会开幕_要闻图片_中国政府网,https://www.gov.cn/yaowen/tupian/202407/conten...,"7月4日7月4日在2024世界人工智能大会特斯拉展位拍摄的人形机器人。 当日,2024世界人..."
67,大会,2,9,中国共产党第二十次全国代表大会在京闭幕 习近平发表重要讲话,https://haokan.baidu.com/v?pd=wisenatural&vid=...,重要讲话 38:35 点击进入沉浸观看 00:00 / 00:00 播放：132万次简介： ...


In [33]:
df.to_csv('baidu.csv', index=False)